In [ ]:


year = 2015

h_mask = rasterio.open(h_mask_loc)
h_mask_data = h_mask.read(1)
meta = h_mask.meta.copy()
meta["nodata"] = 0

start_time = time.time()

iter_subzones = subzones

num_total = len(iter_subzones) * len(all_vars)
num_done = 0

for variable in all_vars:
    clear_output(wait = True)
    print("loading", variable)
    load_start = time.time()
    
    rst_data = load_data(variable, year)      
    
    if variable == "Change_Attribution":
        change_year = load_data("Greatest_Change_Year", year)
        
    load_end = time.time()
    
    print("loading", variable, "took", load_end - load_start)
    
    for soi in iter_subzones:
        
        if variable in struct_vars:
            save_name = soi + "-" + variable + "-" + str(year) + ".csv"
            save_loc = os.path.join("..", "data", "structure", save_name)
        
        elif variable == "Change_Attribution":
            save_name = soi + "-" + variable + ".csv"
            save_loc = os.path.join("..", "data", "disturbance", save_name)
            
        elif variable == "vlce":
            save_name = soi + "-" + variable + "-" + str(year) + ".csv"
            save_loc = os.path.join("..", "data", "vlce", save_name)
            
        elif variable in elev_vars:
            save_name = soi + "-" + variable + ".csv"
            save_loc = os.path.join("..", "data", "elev", save_name)
            
        start_time = time.time()
        
        print(soi, variable, year)
        print(num_done + 1, "/", num_total)
        
        if not os.path.isfile(save_loc):
        
            df = generate_df(soi, variable, year)
            
            if variable == "Change_Attribution":

                df.to_csv(save_loc)

            else:

                df.to_csv(save_loc, index = False)            
                
        
        num_done += 1
        
        end_time = time.time()
        
        print(end_time - start_time)
    
    if 'change_year' in globals():
        del change_year
        
del h_mask
del h_mask_data

In [29]:
# generate new scratch masks that include harvested pixels
h_mask = rasterio.open(h_mask_loc)
h_mask_data = h_mask.read(1)
meta = h_mask.meta.copy()
meta["nodata"] = 0

del h_mask
del h_mask_data

i = 0
num_total = len(subzones)
for soi in subzones:
    print(soi, i + 1, "/", num_total)
    
    subzone_filter = ppa_bec[ppa_bec["szs"] == soi]
    
    # split into protected/non protected shapefiles and save them (not sure its necessary to save)
    # selected protected shapes
    # removes invalid geometries
    p_sub = subzone_filter[subzone_filter["protected"] == True]
    p_sub_clean = p_sub[(p_sub.geometry.type == "Polygon") | (p_sub.geometry.type == "MultiPolygon")]
    p_sub_clean = p_sub_clean.dissolve(by = "szs")

    # select unprotected shapes
    np_sub = subzone_filter[subzone_filter["protected"] == False]
    # removes invalid geometries
    np_sub_clean = np_sub[(np_sub.geometry.type == "Polygon") | (np_sub.geometry.type == "MultiPolygon")]
    np_sub_clean = np_sub_clean.dissolve(by = "szs")
    
    # save locations
    p_raster_loc = os.path.join(scratch, "p-" + soi + ".tif")
    np_raster_loc = os.path.join(scratch, "np-" + soi + ".tif")
    
    # generate masks from the polygons
    p_mask = rasterize_polygon_mask(p_sub_clean, p_raster_loc, meta)
    np_mask = rasterize_polygon_mask(np_sub_clean, np_raster_loc, meta)
    
    i += 1

BAFA_un 1 / 127
BAFA_unp 2 / 127
BG_xh 3 / 127
BG_xw 4 / 127
BWBS_dk 5 / 127
BWBS_mk 6 / 127
BWBS_mw 7 / 127
BWBS_vk 8 / 127
BWBS_wk 9 / 127
CDF_mm 10 / 127
CMA_un 11 / 127
CMA_unp 12 / 127
CMA_wh 13 / 127
CWH_dm 14 / 127
CWH_ds 15 / 127
CWH_mm 16 / 127
CWH_ms 17 / 127
CWH_vh 18 / 127
CWH_vm 19 / 127
CWH_wh 20 / 127
CWH_wm 21 / 127
CWH_ws 22 / 127
CWH_xm 23 / 127
ESSF_dc 24 / 127
ESSF_dcp 25 / 127
ESSF_dcw 26 / 127
ESSF_dk 27 / 127
ESSF_dkp 28 / 127
ESSF_dkw 29 / 127
ESSF_dv 30 / 127
ESSF_dvp 31 / 127
ESSF_dvw 32 / 127
ESSF_mc 33 / 127
ESSF_mcp 34 / 127
ESSF_mh 35 / 127
ESSF_mk 36 / 127
ESSF_mkp 37 / 127
ESSF_mm 38 / 127
ESSF_mmp 39 / 127
ESSF_mmw 40 / 127
ESSF_mv 41 / 127
ESSF_mvp 42 / 127
ESSF_mw 43 / 127
ESSF_mwp 44 / 127
ESSF_mww 45 / 127
ESSF_un 46 / 127
ESSF_unp 47 / 127
ESSF_vc 48 / 127
ESSF_vcp 49 / 127
ESSF_vcw 50 / 127
ESSF_wc 51 / 127
ESSF_wcp 52 / 127
ESSF_wcw 53 / 127
ESSF_wh 54 / 127
ESSF_wk 55 / 127
ESSF_wm 56 / 127
ESSF_wmp 57 / 127
ESSF_wmw 58 / 127
ESSF_wv 59 / 127
ES

In [13]:
# temporal df generation

years = range(1984, 2020)

h_mask = rasterio.open(h_mask_loc)
h_mask_data = h_mask.read(1)
meta = h_mask.meta.copy()
meta["nodata"] = 0

start_time = time.time()

temp_vars = all_vars[0:4]

num_total = len(subzones) * len(temp_vars) * len(years)
num_done = 0

for variable in temp_vars:
    
    load_start = time.time()
    
    rst_data = load_data(variable, year)
    
    load_end = time.time()
    
    print("loading", variable, "took", load_end - load_start)
    
    for year in years:
    
        for soi in subzones:

            if variable in struct_vars:
                save_name = soi + "-" + variable + "-" + str(year) + ".csv"
                save_loc = os.path.join("..", "data", "structure", save_name)

            
            elif variable == "vlce":
                save_name = soi + "-" + variable + "-" + str(year) + ".csv"
                save_loc = os.path.join("..", "data", "vlce", save_name)

            start_time = time.time()

            #
            print(soi, variable, year)
            print(num_done + 1, "/", num_total)

            if not os.path.isfile(save_loc):
                


                df = generate_df(soi, variable, year)

                if variable in struct_vars:

                    df.to_csv(save_loc, index = False)

                elif variable == "vlce":

                    df.to_csv(save_loc, index = False)

                elif variable == "Change_Attribution":

                    df.to_csv(save_loc)
                else:
                    print("not a valid variable")

            num_done += 1

            end_time = time.time()

            print(end_time - start_time)
            clear_output(wait = True)
        
del h_mask
del h_mask_data

SBPS_dc vlce 1985
13951 / 18288


KeyboardInterrupt: 

In [ ]:
def load_data(variable, year):
    if variable in structure_vars:
        file_loc = os.path.join(masked_structure_locs, variable, "masked-" + variable + "-" + str(year) + ".tif")
        
    elif variable == "Change_Attribution" or variable == "Greatest_Change_Year":
        file_loc = os.path.join(merged, variable, "BC-" + variable + ".tif") 
        
    elif variable == "vlce":
        file_loc = os.path.join(merged, variable, "BC-" + variable + "-" + str(year) + ".tif")
        
    elif variable in elev_vars:
        file_loc = os.path.join(merged, "elevation", "BC-" + variable + ".tif")
        
    else:
        print("Not a valid variable")
        return

    with rasterio.open(file_loc) as rst:
        rst_data = rst.read(1)
        
    return rst_data

def get_data(mask, variable, year):
    
    if variable in structure_vars:
        divisor = structure_key[structure_key["variable"] == variable].iloc[0]["divide_by"]
        
    indexes = np.where(mask == 1)
    x = indexes[0]
    y = indexes[1]
    
    data_arr = rst_data[x, y]
    
    if variable in structure_vars:
        # data_arr = data_arr[np.where(data_arr != 0)]
        return data_arr / divisor
    
    elif variable == "Change_Attribution":
        data_year = change_year[x, y]
        
        return data_arr, data_year
    
    else:
        return data_arr
    
def generate_df(soi, variable, year):
    # checks if masks exist for both protected and non protected
    # if it exists, don't generate new ones
    mask_locs = [os.path.join(bec_mask_loc, protected + "-" + soi + "-sampled" + ".tif") for protected in ["p", "np"]]
    mask_sum = sum([os.path.isfile(loc) for loc in mask_locs])

    if mask_sum == 2:
        masks = []

        for i in range(len(mask_locs)):
            with rasterio.open(mask_locs[i]) as rst:
                masks.append(rst.read(1))
    else:
        masks = equal_sample_p_np(soi)
    
    p_data = get_data(masks[0], variable, year)
    np_data = get_data(masks[1], variable, year)
    
    if variable in structure_vars or variable in elev_vars:
        df1 = pd.DataFrame(p_data, columns = ["value"])
        
        df2 = pd.DataFrame(np_data, columns = ["value"])
    
    elif variable == "vlce":
    
        counts = np.transpose(np.unique(p_data, return_counts = True))

        df1 = pd.DataFrame(counts, columns = ["class_val", "cells"])
        
        counts = np.transpose(np.unique(np_data, return_counts = True))
    
        df2 = pd.DataFrame(counts, columns = ["class_val", "cells"])
        
    elif variable == "Change_Attribution":
        df1 = pd.DataFrame(np.transpose(p_data), columns = ["class_val", "year_disturbed"])
        
        df1["n"] = 1
        df1 = df1.groupby(["class_val", "year_disturbed"]).count()[["n"]]
        
        df2 = pd.DataFrame(np.transpose(np_data), columns = ["class_val", "year_disturbed"])
        
        df2["n"] = 1
        df2 = df2.groupby(["class_val", "year_disturbed"]).count()[["n"]]
                           
    else:
        print("Not a valid variable")
        return
        
        
    df1["protected"] = True    
    df2["protected"] = False
    
    df = pd.concat([df1, df2])
    
    df["subzone"] = soi
    df["variable"] = variable
    
    if variable != "Change_Attribution":
        df["year"] = year

    return df

In [9]:
year = 2015

h_mask = rasterio.open(h_mask_loc)
h_mask_data = h_mask.read(1)
meta = h_mask.meta.copy()
meta["nodata"] = 0

start_time = time.time()

iter_subzones = subzones

num_total = len(iter_subzones) * len(all_vars)
num_done = 0

for variable in all_vars:
    clear_output(wait = True)
    print("loading", variable)
    load_start = time.time()
    
    rst_data = load_data(variable, year)      
    
    if variable == "Change_Attribution":
        change_year = load_data("Greatest_Change_Year", year)
        
    load_end = time.time()
    
    print("loading", variable, "took", load_end - load_start)
    
    for soi in iter_subzones:
        
        if variable in structure_vars:
            save_name = soi + "-" + variable + "-" + str(year) + ".csv"
            save_loc = os.path.join("..", "data", "structure", save_name)
        
        elif variable == "Change_Attribution":
            save_name = soi + "-" + variable + ".csv"
            save_loc = os.path.join("..", "data", "disturbance", save_name)
            
        elif variable == "vlce":
            save_name = soi + "-" + variable + "-" + str(year) + ".csv"
            save_loc = os.path.join("..", "data", "vlce", save_name)
            
        elif variable in elev_vars:
            save_name = soi + "-" + variable + ".csv"
            save_loc = os.path.join("..", "data", "elev", save_name)
            
        start_time = time.time()
        
        print(soi, variable, year)
        print(num_done + 1, "/", num_total)
        
        if not os.path.isfile(save_loc):
        
            df = generate_df(soi, variable, year)
            
            if variable == "Change_Attribution":

                df.to_csv(save_loc)

            else:

                df.to_csv(save_loc, index = False)            
                
        
        num_done += 1
        
        end_time = time.time()
        
        print(end_time - start_time)
    
    if 'change_year' in globals():
        del change_year
        
del h_mask
del h_mask_data

loading Change_Attribution
loading Change_Attribution took 55.01481485366821
BAFA_un Change_Attribution 2015
763 / 889
0.0010094642639160156
BAFA_unp Change_Attribution 2015
764 / 889
0.0020046234130859375
BG_xh Change_Attribution 2015
765 / 889
0.0009834766387939453
BG_xw Change_Attribution 2015
766 / 889
0.0009989738464355469
BWBS_dk Change_Attribution 2015
767 / 889
0.0009989738464355469
BWBS_mk Change_Attribution 2015
768 / 889
0.0019991397857666016
BWBS_mw Change_Attribution 2015
769 / 889
0.00099945068359375
BWBS_vk Change_Attribution 2015
770 / 889
0.0010004043579101562
BWBS_wk Change_Attribution 2015
771 / 889
0.00099945068359375
CDF_mm Change_Attribution 2015
772 / 889
0.00099945068359375
CMA_un Change_Attribution 2015
773 / 889
0.0009999275207519531
CMA_unp Change_Attribution 2015
774 / 889
0.000997781753540039
CMA_wh Change_Attribution 2015
775 / 889
0.0009996891021728516
CWH_dm Change_Attribution 2015
776 / 889
0.0009992122650146484
CWH_ds Change_Attribution 2015
777 / 889
